In [1]:
!pip install datasets transformers -qqq

In [111]:
from transformers import AutoTokenizer, T5TokenizerFast
model_checkpoint = "t5-base"
prefix = "translate English to Sparql: "
model_path='../../../data/models/sparql-translator-t5-2021-07-09'
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint, src_lang="en", tgt_lang="sparql")

In [112]:
with tokenizer.as_target_tokenizer():
      print(tokenizer('Allooo', max_length=100, truncation=True))

{'input_ids': [432, 32, 32, 32, 1], 'attention_mask': [1, 1, 1, 1, 1]}


In [4]:
# from google.colab import drive
# drive.mount('drive')

In [113]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)#.to("cuda")

In [114]:
from transformers import pipeline

translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer
)

In [1]:
import pandas as pd
import re

In [28]:
df_item = pd.read_csv('../../../data/kdwd/item_no_desc.csv')#.set_index('item_id')
df_property = pd.read_csv('../../../data/kdwd/property.csv')#.set_index('property_id')
del df_property['en_description']
# del df_item['Unnamed: 0']

In [4]:
# Lower Prop Dataset
df_property['en_label'] = df_property['en_label'].str.lower()

In [ ]:
df_item_aliases = pd.read_csv('../../../data/kdwd/item_aliases.csv')#.set_index('item_id')
df_property_aliases = pd.read_csv('../../../data/kdwd/property_aliases.csv')#.set_index('property_id')
df_item_aliases['en_alias'] = df_item_aliases['en_alias'].str.lower()
df_property_aliases['en_alias'] = df_property_aliases['en_alias'].str.lower()

In [6]:
def encode_props(qry):
    # Q
    for m in re.finditer(":Q\d+", qry):
        x = m.group(0)[1:]
        newstring = df_item.loc[int(x[1:])]['en_label'].replace(" ", "_").lower()
        qry = qry.replace(x, newstring)
    # P
    for m in re.finditer(":P\d+", qry):
        x = m.group(0)[1:]
        newstring = df_property.loc[int(x[1:])]['en_label'].replace(" ", "_").lower()
        qry = qry.replace(x, newstring)
    return qry
encode_props('SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),\'2009\')) }')
# "select ?obj where [ wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]"

"SELECT ?obj WHERE { wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) }"

In [123]:
def replace_all(text, dict):
    for i, j in dict.items(): text = text.replace(i, j)
    return text

In [222]:
def decode_props(qry):
    rep_dict = {}
    qry = qry.replace('[','{').replace(']','}')
    for m in re.finditer(":\w+", qry):
        pref = qry[m.start(0)-2:m.start(0)]
        try:
            x = m.group(0)[1:]
            if 'wd' not in pref:
                s = df_property[df_property['en_label']==x.replace("_", " ")]
                if len(s)==0:
                    s = df_property_aliases[df_property_aliases['en_alias']==x.replace("_", " ")]
                if len(s)>0:
                    id = s.iloc[0][0]
                    rep_dict[x]='P'+str(id)
                    continue
            # Use else to do not replace missing Ps with Qs
            s = df_item[df_item['en_label']==x.replace("_", " ")]
            if len(s)==0:
                s = df_item_aliases[df_item_aliases['en_alias']==x.replace("_", " ")]
            id = s.iloc[0][0]
            rep_dict[x]='Q'+str(id)
        except: print(m)
    return (qry, replace_all(qry, rep_dict))

In [204]:
# rplc_props('SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),\'2009\')) }')
decode_props("select ?obj where [ wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]")

("select ?obj where { wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) }",
 "select ?obj where { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }")

In [194]:
translate= lambda q: decode_props(translator(prefix+q, max_length=100)[0]['translation_text'])

In [172]:
!pip install beautifultable -qqq

In [195]:
def pretty_translate(t,q): 
    ans = translate(q)
    # print('QUESTION:\n' + q + '\nTARGET:\n'+ t +'\nRESULT:\n' + ans[0]+ '\nRESULT-DECODED:\n' + ans[1])
    from beautifultable import BeautifulTable
    table = BeautifulTable(maxwidth=140)
    # table.column_headers = ["", "Man Utd","Man City","T Hotspur"]
    table.rows.append(['QUESTION', q])
    table.rows.append(['Target', t])
    table.rows.append(['RESULT-DECODED', ans[1]])
    table.rows.append(['RESULT-Raw', ans[0]])
    print(table)

# TESTING

In [152]:
translate('Who is Marlin Manson?')

Generated by the model:
select distinct ?sbj where [ ?sbj wdt:instance_of wd:marlin_manson . ?sbj wdt:instance_of wd:human ]
<re.Match object; span=(52, 66), match=':marlin_manson'>


'select distinct ?sbj where { ?sbj wdt:P31 wd:marlin_manson . ?sbj wdt:P31 wd:Q5 }'

In [153]:
translate('Who is Bill Gates?')

Generated by the model:
select distinct ?sbj where [ ?sbj wdt:instance_of wd:bill_gates . ?sbj wdt:instance_of wd:human ]


'select distinct ?sbj where { ?sbj wdt:P31 wd:Q5284 . ?sbj wdt:P31 wd:Q5 }'

In [148]:
translate('What is the time zone of Salt Lake City?')

Generated by the model:
select ?answer where [ wd:salt_lake_city wdt:located_in_time_zone ?X . ?X wdt:location ?answer]


'select ?answer where { wd:Q23337 wdt:P421 ?X . ?X wdt:P276 ?answer}'

In [196]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+----------------+----------------------------------------------------------------------------------------------------------+
|    QUESTION    |                          Who is the country for head of state of Mahmoud Abbas?                          |
+----------------+----------------------------------------------------------------------------------------------------------+
|     Target     |              select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
| RESULT-DECODED |              select distinct ?obj where { wd:Q127998 wdt:P35 ?obj . ?obj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
|   RESULT-Raw   | select distinct ?obj where { wd:mahmoud_abbas wdt:head_of_state ?obj . ?obj wdt:instance_of wd:coun

In [197]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }',"What country is Mahmoud Abbas the head of state of?")

+----------------+----------------------------------------------------------------------------------------------------------+
|    QUESTION    |                           What country is Mahmoud Abbas the head of state of?                            |
+----------------+----------------------------------------------------------------------------------------------------------+
|     Target     |              select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
| RESULT-DECODED |              select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
|   RESULT-Raw   | select distinct ?sbj where { ?sbj wdt:head_of_state wd:mahmoud_abbas . ?sbj wdt:instance_of wd:coun

In [198]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+----------------+----------------------------------------------------------------------------------------------------------+
|    QUESTION    |                          Who is the country for head of state of Mahmoud Abbas?                          |
+----------------+----------------------------------------------------------------------------------------------------------+
|     Target     |              select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
| RESULT-DECODED |              select distinct ?obj where { wd:Q127998 wdt:P35 ?obj . ?obj wdt:P31 wd:Q6256 }              |
+----------------+----------------------------------------------------------------------------------------------------------+
|   RESULT-Raw   | select distinct ?obj where { wd:mahmoud_abbas wdt:head_of_state ?obj . ?obj wdt:instance_of wd:coun

In [223]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress is the voice over on South Park and is employed as a singer?".lower())

+----------------+-----------------------------------------------------------------------------------------------+
|    QUESTION    |       which female actress is the voice over on south park and is employed as a singer?       |
+----------------+-----------------------------------------------------------------------------------------------+
|     Target     |       SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}        |
+----------------+-----------------------------------------------------------------------------------------------+
| RESULT-DECODED |     select ?answer where { wd:Q16538 wdt:Q12280274 ?answer . ?answer wdt:P108 wd:Q177220}     |
+----------------+-----------------------------------------------------------------------------------------------+
|   RESULT-Raw   | select ?answer where { wd:south_park wdt:voice_over ?answer . ?answer wdt:employer wd:singer} |
+----------------+--------------------------------------------------------------

In [224]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress on South Park is the voice over and is used as a singer?")

+----------------+-----------------------------------------------------------------------------------+
|    QUESTION    |   Which female actress on South Park is the voice over and is used as a singer?   |
+----------------+-----------------------------------------------------------------------------------+
|     Target     | SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}  |
+----------------+-----------------------------------------------------------------------------------+
| RESULT-DECODED |   select ?answer where { wd:Q16538 wdt:Q12280274 ?answer . ?answer wdt:Q177220}   |
+----------------+-----------------------------------------------------------------------------------+
|   RESULT-Raw   | select ?answer where { wd:south_park wdt:voice_over ?answer . ?answer wdt:singer} |
+----------------+-----------------------------------------------------------------------------------+


In [225]:
# Paul Erdős (Q173746)
pretty_translate("select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}","Which is the PIM authority ID of Paul Erd?")

<re.Match object; span=(25, 34), match=':paul_erd'>
+----------------+-----------------------------------------------------------------------------------+
|    QUESTION    |                    Which is the PIM authority ID of Paul Erd?                     |
+----------------+-----------------------------------------------------------------------------------+
|     Target     |           select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}           |
+----------------+-----------------------------------------------------------------------------------+
| RESULT-DECODED |       select ?answer where { wd:paul_erd wdt:P797 ?X . ?X wdt:P31 ?answer}        |
+----------------+-----------------------------------------------------------------------------------+
|   RESULT-Raw   | select ?answer where { wd:paul_erd wdt:authority ?X . ?X wdt:instance_of ?answer} |
+----------------+-----------------------------------------------------------------------------------+


In [226]:
pretty_translate("SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }", 
            "What was the population of Somalia in 2009-0-0?")

+----------------+-------------------------------------------------------------------------------------------------------------------------+
|    QUESTION    |                                     What was the population of Somalia in 2009-0-0?                                     |
+----------------+-------------------------------------------------------------------------------------------------------------------------+
|     Target     |     SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }      |
+----------------+-------------------------------------------------------------------------------------------------------------------------+
| RESULT-DECODED |     select ?obj where { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }      |
+----------------+-------------------------------------------------------------------------------------------------------------------------+
|   RESULT-Ra

In [229]:
translate('Humans born in New York City') #random query - answer seems correct

('select distinct ?sbj where { ?sbj wdt:place_of_birth wd:new_york_city . ?sbj wdt:instance_of wd:human }',
 'select distinct ?sbj where { ?sbj wdt:P19 wd:Q60 . ?sbj wdt:P31 wd:Q5 }')